# AiDM Assignment 3 Group 81
### Meng Yao (yao@strw.leidenuniv.nl) and Michael Keim (keim@strw.leidenuniv.nl)
##  Part 3: Intial PageRank Implementation

In [9]:
import numpy as np
from scipy.sparse import csc_matrix

First we must load IDs of page link origins and targets and all unqiue IDS generated in prep.ipynb

In [10]:
ID_From = np.load('ID_From.npy')
ID_To = np.load('ID_To.npy')

Next we must find unique page IDs. These pages will be both the rows and columns of our matrix M which we will use to simulate random walks across nodes.

In [11]:
ID_Unique = np.unique(np.concatenate((ID_From, ID_To), axis=0))

Now we count nonzero outgoing links. This will serve as the denominator for the columns of our matrix M, making an even distribution 1 / (the total outgoing count) for connected nodes as the proability of going to the page of a row from the page of a column

In [12]:
ID_Outgoing_Nonzero, ID_Outgoing_Count = np.unique(ID_From, return_counts=True)

Now we create our matrix M as a sparse matrix. We initalize with ones (next step is normalization) in the rows and columns of destination and current page nodes as specified in ID_To and ID_From.

In [13]:
M = csc_matrix((np.ones(len(ID_From)), (ID_To, ID_From)), shape=(len(ID_Unique), len(ID_Unique)))

Now we normalize by dividing columns the total number of outgoing node connections. We will use 1 if the node is a dead end. The result of the division will still be 0, as there would be no entries for the column in our sparse matrix, but we will avoid nans by doing so.

In [14]:
Norm = np.ones(len(ID_Unique))
Norm[ID_Outgoing_Nonzero] = ID_Outgoing_Count
val = np.repeat(Norm, M.getnnz(axis=0))
M.data /= val

Next we create a vector v with an entry for each page and an even inital distribution across all pages. In the end this will measure the page importance if we properly use the PageRank update rule and converge.

In [15]:
v = np.ones(len(ID_Unique))/len(ID_Unique)

Here we create a vector to represent teleportation, a method to attempt to correct for dead ends.

In [16]:
beta = 0.8
v_teleport = (1-beta)*np.ones(len(ID_Unique))/len(ID_Unique)

Finally we iterate to creat v_new, which converges fairly quickly!

In [17]:
iterations = 25
v_new = np.zeros(len(ID_Unique))
MSE = np.zeros(iterations) # To store error
for i in range(iterations):
        v_new = beta * M * v + v_teleport
        MSE[i] = np.sum((v - M * v)**(2.))
        v = v_new
print(MSE)

[2.04970154e-03 4.74910361e-04 3.16997638e-05 8.76108822e-06
 1.12268880e-05 1.16729133e-05 1.20460437e-05 1.21650927e-05
 1.22272553e-05 1.22559916e-05 1.22724479e-05 1.22818350e-05
 1.22881334e-05 1.22921225e-05 1.22950448e-05 1.22970072e-05
 1.22985003e-05 1.22995409e-05 1.23003446e-05 1.23009196e-05
 1.23013663e-05 1.23016921e-05 1.23019456e-05 1.23021334e-05
 1.23022796e-05]
